# Evaluation Example 2 - Daily Data, CAMELS Subset (542)
#####

Example 2 walks through an anticipated common use case of evaluating experimental model output at a large sample of gauge locations using TEEHR. We are using MARRMoT model 37 (HBV) output as our ‘experimental’ model. (Thank you, Wouter - source: Knoben et al 2019) and a subset of CAMELS basins (543 of the 671).

#### In this notebook we will perform the following steps:
<ol>
    <li> Get the TEEHR datasets from S3</li>
    <li> Build a joined duckdb database</li> 
    <li> Add attributes to the joined database</li>
    <li> Add calculated fields to the joined database</li>
    <li> Export the joined duckdb database to a parquet file</li>
</ol>

## 1. Get the data from S3
For the sake of time, we prepared the individual datasets in advance and are simply copying to your 2i2c home directory. After running the cell below to copy the example_2 data.

In [62]:
!rm -rf ~/teehr/example-2/*
!aws s3 cp --recursive s3://ciroh-rti-public-data/teehr-workshop-devcon-2024/workshop-data/example-2 ~/teehr/example-2

download: s3://ciroh-rti-public-data/teehr-workshop-devcon-2024/workshop-data/example-2/attributes/usgs_basin_attr_aridity.camels.parquet to ../../home/jovyan/teehr/example-2/attributes/usgs_basin_attr_aridity.camels.parquet
download: s3://ciroh-rti-public-data/teehr-workshop-devcon-2024/workshop-data/example-2/attributes/usgs_basin_attr_drainage_area.all.parquet to ../../home/jovyan/teehr/example-2/attributes/usgs_basin_attr_drainage_area.all.parquet
download: s3://ciroh-rti-public-data/teehr-workshop-devcon-2024/workshop-data/example-2/attributes/usgs_basin_attr_frac_snow.camels.parquet to ../../home/jovyan/teehr/example-2/attributes/usgs_basin_attr_frac_snow.camels.parquet
download: s3://ciroh-rti-public-data/teehr-workshop-devcon-2024/workshop-data/example-2/attributes/usgs_point_attr_q_mean.camels.parquet to ../../home/jovyan/teehr/example-2/attributes/usgs_point_attr_q_mean.camels.parquet
download: s3://ciroh-rti-public-data/teehr-workshop-devcon-2024/workshop-data/example-2/attr

In [63]:
!tree ~/teehr/example-2/

/home/jovyan/teehr/example-2/
├── attributes
│   ├── usgs_basin_attr_aridity.camels.parquet
│   ├── usgs_basin_attr_drainage_area.all.parquet
│   ├── usgs_basin_attr_drainage_area.gages2.parquet
│   ├── usgs_basin_attr_forest_frac.camels.parquet
│   ├── usgs_basin_attr_frac_snow.camels.parquet
│   ├── usgs_basin_attr_frac_urban.conus.parquet
│   ├── usgs_point_attr_baseflow_index.camels.parquet
│   ├── usgs_point_attr_ecoregions.conus.parquet
│   ├── usgs_point_attr_q5.camels.parquet
│   ├── usgs_point_attr_q95.camels.parquet
│   ├── usgs_point_attr_q_mean.camels.parquet
│   ├── usgs_point_attr_rfc.conus.parquet
│   ├── usgs_point_attr_runoff_ratio.camels.parquet
│   ├── usgs_point_attr_slope_fdc.camels.parquet
│   └── usgs_point_attr_stream_order.conus.parquet
├── crosswalks
│   ├── usgs_nwm30_crosswalk.conus.parquet
│   └── usgs_usgs_crosswalk.conus.parquet
├── geometry
│   └── usgs_points.parquet
├── primary
│   └── usgs_daily_mean_1989_2009.parquet
└── secondary
    ├── marrmot37_h

## 2. Build the joined database
An essential step for all model evalutions is joining the dataset being evaluated with a dataset that is considered 'truth' - i.e., the verifying data (typically some form of observations). What we mean by 'joining' is aligning a model output data (location and time) with the observations at the same (or most representative possible) location and time.  If the evaluation includes a small number of locations and/or a small period of time (total amount of data is relatively small), the joining process could be done on the fly when calculating metrics.  However as the magnitude of data becomes very large (e.g., 1000s of locations, 20+ years of hourly data), joining data on the fly and/or storing all of the joined data in memory can become prohibitively slow or infeasible, depending on your available memory.  Further if including metric uncertainty bounds (to be added in future versions of TEEHR), the joined dataset must be resampled, creating yet more data to handle in memory.  To address this, TEEHR creates a joined dataset in advance and writes the results to disk in highly efficient parquet format. For context, the 44-year NWM retrospective simulations at ~8000 gauge locations results in X billion rows of data.  Joining this data on the fly would be not be feasible on most servers.

The TEEHR class ```DuckDBDatabase``` of module ```teehr.classes.duckdb_database``` provides methods to create a DuckDB database, insert joined time series, attributes (Step 3) and other calculated fields (Step 4) useful for evalution.  We can then export to a parquet file so we can store it anywhere (S3) and efficiently test different metrics, grouping and filtering approaches for evaluation. 

The arguments to build a joined database using ```insert_joined_timeseries``` on the  ```DuckDBDatabase``` class are:

<ul>
    <li> primary_filepath -> filepath of parquet file(s) containing the primary (observed) time series </li>
    <li> secondary_filepath -> filepath of parquet file(s) containing the secondary (1 or more model/baseline) time series </li>
    <li> crosswalk_filepath -> filepath of parquet file(s) containing crosswalks between primary and secondary location IDs </li>
    <li> order_by -> list of fields to sort the data in the database</li>
</ul>

In [64]:
JOINED_FILEPATH.parent

PosixPath('/home/jovyan/teehr/example-2/joined')

In [65]:
%%time

from teehr.classes.duckdb_database import DuckDBDatabase
from pathlib import Path

TEEHR_BASE = Path(Path.home(), "teehr/example-2")
PRIMARY_FILEPATH = f"{TEEHR_BASE}/primary/**/*.parquet"
SECONDARY_FILEPATH = f"{TEEHR_BASE}/secondary/**/*.parquet"
CROSSWALK_FILEPATH = f"{TEEHR_BASE}/crosswalks/**/*.parquet"
ATTRIBUTE_FILEPATH = f"{TEEHR_BASE}/attributes/**/*.parquet"

# define the joined parquet filepath and create parent directory
JOINED_FILEPATH = Path(TEEHR_BASE, "joined", "teehr_joined.parquet")
JOINED_FILEPATH.parent.mkdir(exist_ok=True, parents=True)

# temporary DuckDB database that will be exported to parquet
DB_FILEPATH = Path(JOINED_FILEPATH.parent, "teehr.db")

# if the database already exists, remove it first
if Path(DB_FILEPATH).is_file():
   Path(DB_FILEPATH).unlink()

# create the database and insert timeseries
ddd = DuckDBDatabase(f"{DB_FILEPATH}")
ddd.insert_joined_timeseries(
    primary_filepath=PRIMARY_FILEPATH,
    secondary_filepath=SECONDARY_FILEPATH,
    crosswalk_filepath=CROSSWALK_FILEPATH,
    order_by=[
        "primary_location_id",
        "configuration",
        "value_time"
    ],
)    

# confirm fields in the DB
ddd.get_joined_timeseries_schema()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 47.2 s, sys: 7.52 s, total: 54.7 s
Wall time: 14.9 s


,column_name,column_type,null,key,default,extra
0,reference_time,TIMESTAMP,YES,None,None,None
1,value_time,TIMESTAMP,YES,None,None,None
2,secondary_location_id,VARCHAR,YES,None,None,None
3,secondary_value,FLOAT,YES,None,None,None
4,configuration,VARCHAR,YES,None,None,None
5,measurement_unit,VARCHAR,YES,None,None,None
6,variable_name,VARCHAR,YES,None,None,None
7,primary_value,FLOAT,YES,None,None,None
8,primary_location_id,VARCHAR,YES,None,None,None


### 4. Add attributes

In this step, we will add those attributes to the joined database. Attributes add more “power” to the evaluation - i.e., more ways to group and filter the data for metric calculations, more complex metrics, and more insightful visualizations

The arguments to  using ```insert_attributes``` on the  ```DuckDBDatabase``` class are:

<ul>
    <li> attribute_filepath -> filepath of parquet file(s) containing the attribute data by the primary location ID</li>
</ul>

In [66]:
%%time
ddd.insert_attributes(attributes_filepath=ATTRIBUTE_FILEPATH)

# confirm fields in now the DB
ddd.get_joined_timeseries_schema()

CPU times: user 33.1 s, sys: 1.65 s, total: 34.8 s
Wall time: 30.5 s


,column_name,column_type,null,key,default,extra
0,reference_time,TIMESTAMP,YES,None,None,None
1,value_time,TIMESTAMP,YES,None,None,None
2,secondary_location_id,VARCHAR,YES,None,None,None
3,secondary_value,FLOAT,YES,None,None,None
4,configuration,VARCHAR,YES,None,None,None
5,measurement_unit,VARCHAR,YES,None,None,None
6,variable_name,VARCHAR,YES,None,None,None
7,primary_value,FLOAT,YES,None,None,None
8,primary_location_id,VARCHAR,YES,None,None,None
9,q95_cms,VARCHAR,YES,None,None,None


### 5. Add calculated fields to the pre-joined duckdb database

Calculated fields open up an even wider range of options for evaluating your data.  It allows you to write simple functions to calculate new fields based on any existing fields (data or attributes), which can then be used to group or filter data for metric calculations.

In the examples below, two calculated field are added to the database:

<ul>
    <li> the month -> to enable calculating metrics for specific months or ranges of months (season) </li>
    <li> flow category relative to a threshold (above or below) -> to enable calcualting metrics for varying definitions of high or low flow </li>
</ul>



In [67]:
# Add calculated fields
import datetime as datetime

# Function arguments should have the same data type as the fields used. 
# Note: In the data model, all attribute values are added to the db as type 'str' 

def add_month_field(arg1: datetime) -> int:
    return arg1.month

def add_flow_category_relative_to_threshold_field(arg1: float, arg2: str) -> str:
    if arg1 >= float(arg2):
        return 'high'
    else:
        return 'low'

# month
ddd.insert_calculated_field(new_field_name="month",
                    new_field_type="INTEGER",
                    parameter_names=["value_time"],
                    user_defined_function=add_month_field)

# obs above mean
ddd.insert_calculated_field(new_field_name="obs_flow_category_q_mean",
                    new_field_type="VARCHAR",
                    parameter_names=["primary_value", "q_mean_cms"],
                    user_defined_function=add_flow_category_relative_to_threshold_field)

# view fields now in the DB
ddd.get_joined_timeseries_schema()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,column_name,column_type,null,key,default,extra
0,reference_time,TIMESTAMP,YES,None,None,None
1,value_time,TIMESTAMP,YES,None,None,None
2,secondary_location_id,VARCHAR,YES,None,None,None
3,secondary_value,FLOAT,YES,None,None,None
4,configuration,VARCHAR,YES,None,None,None
5,measurement_unit,VARCHAR,YES,None,None,None
6,variable_name,VARCHAR,YES,None,None,None
7,primary_value,FLOAT,YES,None,None,None
8,primary_location_id,VARCHAR,YES,None,None,None
9,q95_cms,VARCHAR,YES,None,None,None


### 6. Export the pre-joined duckdb database to a parquet file

In [68]:
import pandas as pd

# generate joined parquet file
ddd.query(f"""
    COPY (
        SELECT * 
        FROM joined_timeseries
        ORDER BY primary_location_id, value_time
    ) 
    TO '{JOINED_FILEPATH}' (FORMAT PARQUET);
""")

# read the resulting parquet file in pandas to check it out
df = pd.read_parquet(JOINED_FILEPATH)
df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,reference_time,value_time,secondary_location_id,secondary_value,configuration,measurement_unit,variable_name,primary_value,primary_location_id,q95_cms,...,stream_order,baseflow_index,frac_urban,q5_cms,q_mean_cms,slope_fdc,frac_snow,upstream_area_km2,month,obs_flow_category_q_mean
0,NaT,1990-10-01,usgs-01013500,23.583260,marrmot_hbv,cms,streamflow_daily_mean,23.419521,usgs-01013500,166.7828551043532,...,5,0.585225955779508,0.01,6.309780816831367,44.467109455834866,1.52821853538976,0.313440357191799,None,10,low
1,NaT,1990-10-01,nwm30-724696,39.858749,nwm30_retro,cms,streamflow_daily_mean,23.419521,usgs-01013500,166.7828551043532,...,5,0.585225955779508,0.01,6.309780816831367,44.467109455834866,1.52821853538976,0.313440357191799,None,10,low
2,NaT,1990-10-02,usgs-01013500,25.138947,marrmot_hbv,cms,streamflow_daily_mean,25.049789,usgs-01013500,166.7828551043532,...,5,0.585225955779508,0.01,6.309780816831367,44.467109455834866,1.52821853538976,0.313440357191799,None,10,low
3,NaT,1990-10-02,nwm30-724696,43.503750,nwm30_retro,cms,streamflow_daily_mean,25.049789,usgs-01013500,166.7828551043532,...,5,0.585225955779508,0.01,6.309780816831367,44.467109455834866,1.52821853538976,0.313440357191799,None,10,low
4,NaT,1990-10-03,usgs-01013500,26.920364,marrmot_hbv,cms,streamflow_daily_mean,26.965897,usgs-01013500,166.7828551043532,...,5,0.585225955779508,0.01,6.309780816831367,44.467109455834866,1.52821853538976,0.313440357191799,None,10,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6445815,NaT,2009-12-29,nwm30-23894572,11.936666,nwm30_retro,cms,streamflow_daily_mean,9.635986,usgs-14316700,72.4911272755203,...,5,0.508616082222394,0.0,0.9627727841279988,19.909239533259516,2.23102329485064,0.176336580742005,None,12,low
6445816,NaT,2009-12-30,usgs-14316700,15.432463,marrmot_hbv,cms,streamflow_daily_mean,9.856622,usgs-14316700,72.4911272755203,...,5,0.508616082222394,0.0,0.9627727841279988,19.909239533259516,2.23102329485064,0.176336580742005,None,12,low
6445817,NaT,2009-12-30,nwm30-23894572,12.146250,nwm30_retro,cms,streamflow_daily_mean,9.856622,usgs-14316700,72.4911272755203,...,5,0.508616082222394,0.0,0.9627727841279988,19.909239533259516,2.23102329485064,0.176336580742005,None,12,low
6445818,NaT,2009-12-31,usgs-14316700,52.657688,marrmot_hbv,cms,streamflow_daily_mean,14.975918,usgs-14316700,72.4911272755203,...,5,0.508616082222394,0.0,0.9627727841279988,19.909239533259516,2.23102329485064,0.176336580742005,None,12,low


In [69]:
# if you are confident the parquet file was successfully created, delete the temporarky DuckDB file
Path(DB_FILEPATH).unlink()